# Problem (from notebooks/api_search_recs_pkl_parquet_sqlite)
- search_recs write/read with either .sqlite/.parquet is fast
- But I forgot to account for the _~20s_ it takes to convert back _to_ search_recs from the serialized df

# Solution: save/load separate files
- feats.npy for heavy np.array cols + non_feats.parquet for normal df cols
- A little more complexity to manage, but an ok tradeoff
- Conclusion: ~1s read, ~3s write

In [ ]:
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs.cache_clear()

INFO     [18:18:15.112] [ 2144]  52 logging_/init_logging: {logging_yaml: None}

INFO     [18:18:15.494] [ 2144]  32 api.server_globals/init

DEBUG    [18:18:15.500] [ 2144]  75 api.server_globals/load: load_search...

INFO     [18:18:15.757] [ 2144]  79 api.server_globals/load: load_search (took 0.252s)

DEBUG    [18:18:15.764] [ 2144]  75 api.server_globals/load: load_xc_meta...

...

DEBUG    [18:18:16.067] [ 2144]  75 api.server_globals/load: load_feat_info...

INFO     [18:18:16.075] [ 2144] 110 api.server_globals/load_feat_info

INFO     [18:18:16.081] [ 2144]  79 api.server_globals/load: load_feat_info (took 0.006s)

WARNING  [18:18:16.088] [ 2144]  73 api.server_globals/load: load_search_recs [skipped]

INFO     [18:18:16.094] [ 2144]  34 api.server_globals/init: done

In [ ]:
kwargss = [
    dict(cache_type='hybrid', refresh=True),  # Simulate cache miss
    dict(cache_type='hybrid'),                # Simulate cache hit
]
search_recss = []
for kwargs in kwargss:
    print()
    print(f'search_recs(**{kwargs})')
    get_search_recs.cache_clear()
    search_recs = get_search_recs(**kwargs)
    search_recss.append(search_recs)
    display(search_recs
        [['xc_id', 'feat', 'f_preds', 'background', 'background_species']]
        [2:3].reset_index(drop=True).T
        .pipe(df_assign_first,
            size=lambda df: df.T.memory_usage(deep=True),
            type=lambda df: df[0].map(lambda x: type(x).__name__),
        )
    )

search_recs(**{'cache_type': 'hybrid', 'refresh': True})

INFO     [18:18:16.252] [ 2144] 305 api.recs/get_search_recs

INFO     [18:18:16.265] [ 2144]  61 payloads/df_cache_hybrid: Miss: payloads/search_recs-version[0],limit[10],audio_s[10],scale[1],countries_k[na],com_names_k[ca],num_recs[None]-84c26f9

df_cache_hybrid:compute[search_recs]:   0%|                                                      | 0/1 [00:00<?, ?it/s]

INFO     [18:18:16.274] [ 2144] 362 api.recs/_compute_search_recs: {len(sg.xc_meta): 35233, countries_k: na, com_names_k: ca, num_recs: null}

...

INFO     [18:18:20.014] [ 2144] 144 payloads/df_cache_hybrid: Hit: Read feat-feat.npy: float32 (60.1 kB)

INFO     [18:18:20.020] [ 2144] 144 payloads/df_cache_hybrid: Hit: Read feat-f_preds.npy: float32 (13.4 kB)

INFO     [18:18:20.026] [ 2144] 148 payloads/df_cache_hybrid: Hit: Join non_feats + feats

INFO     [18:18:20.049] [ 2144] 162 payloads/df_cache_hybrid: Hit: Done (took: 0.370s)

,size,type,0
xc_id,36,int,417610
feat,104,ndarray,[ 6.140e-02 -2.948e-02 1.953e-02 -3.798e-02 2.723e-02 -2.374e-03 5.391e-02 -5.180e-02 1.074e-01 -2.686e-02 -1.881e-01 9.991e-02 -1.477e-02 -7.756e-02 9.021e-02 -1.45...
f_preds,104,ndarray,[2.549e-03 1.458e-03 1.498e-04 1.324e-03 6.637e-04 2.818e-03 3.106e-03 1.472e-03 2.511e-03 4.366e-03 3.564e-04 9.183e-04 5.002e-03 9.269e-03 4.268e-03 2.612e-03 3.691e-04 ...
background,80,list,"['Chipping Sparrow (Spizella passerina)', 'Black-backed Woodpecker (Picoides arcticus)', ""Audubon's Warbler (Setophaga auduboni)"", 'Hermit Warbler (Setophaga occidentalis)'..."
background_species,80,list,"['CHSP', 'BBWO', ""Audubon's Warbler"", 'HEWA']"


In [ ]:
# TODO Turn this into unit tests
#   - Should roundtrip, should preserve category dtypes, should fail if index present
[a, b] = search_recss
pd.testing.assert_frame_equal(a, b, check_column_type=True,
    check_index_type=False,  # Allow Int64Index vs. RangeIndex
)